In [2]:
import urllib.request as ur # import para obtener la página web 
import re                   # import para utilizar expresiones regulares 

# Obtener página web con incidencias de Madrid 
f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=13&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')
s = f.read().decode()
f.close()


In [3]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [10]:
from pprint import pprint     # import para hacer prints boninitos 

'''
Incidencias de la M-40 
'''
carretera = "M-40"

# Obtener las incidencias que tienen hora fin 
print("Incidencias con hora fin")
confin = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(M-40)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(confin)

# Obtener todas las incidencias 
print("\nTodas las incidencias")
#conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(M-40)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
conjunto= re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<td.*?<b>(M-40)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)

pprint(conjunto)

#conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d)?.*?(\d\d/\d\d/\d\d\d\d)?.*?<td.*?<td.*?<td.*?<b>(M-40)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)



Incidencias con hora fin
[('18:44',
  '28/02/2019',
  '18:43',
  '28/02/2019',
  'M-40',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCULACIÓN</b> con circulación '
  'irregular  en<b></b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  -  La carretera  <b> <span style="color:#ab3000">M-40 </span> '
  '</b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 54</b> al <b> km '
  '60</b> sentido <b>DECRECIENTE DE LA KILOMETRACIÓN'),
 ('18:03',
  '28/02/2019',
  '18:02',
  '28/02/2019',
  'M-40',
  'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por <b>CIRCULACIÓN</b> en<b> '
  'Túneles del Pardo</b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  -  La carretera  <b> <span style="color:#ab3000">M-40 </span> '
  '</b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 57</b> al <b> km '
  '46</b> sentido <b>DECRECIENTE DE LA KILOMETRACIÓN</b> hacia <b>Decreciente'),
 ('18:00',
  '28/02/2019',
  '21:00',
  '28/02/2019',
  'M-40',
  'RESTRICCIONES A V

In [11]:
diccionario = {} # variable que se utilizará para almacenar las incidencias una sola vez 

for x in range(len(confin)):
    identificador = confin[x][0] + confin[x][1] + confin[x][4]  # concatenación de valores para crear identificador para diccionario
    diccionario.update({identificador : confin[x]}) # agregación de valores al diccionario

for x in range(len(conjunto)):
    identificador = conjunto[x][0] + conjunto[x][1] + conjunto[x][4] # concatenación de valores para crear identificador para diccionario
    if diccionario.get(identificador) == None:    # En caso de que el identificador exista, salta este paso
        diccionario.update({identificador : conjunto[x]}) # Si el identificador no existe, se crea un nuevo elemento en el diccionario
        
print("diccionario")
pprint(diccionario)
    
for x in diccionario:
    valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
    fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
    cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
    if valores[2] != '':  # En caso de que exista una hora fin
        mensaje = "Hubo un atasco en la carretera " + valores[4]
        fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
    else: 
        mensaje = "Existe un atasco en la carretera " + valores[4]
    mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
    mensaje = mensaje + fin
    mensaje = mensaje + " con la siguientes características:\n" +  cleantext
    


IndexError: tuple index out of range